In [1]:
from app.services.document_conversion_service import document_conversion_service
from app.services.chunking_service import chunking_service
from app.services.context_generation_service import context_generation_service
from app.services.embedding_service import embedding_service
from app.services.metadata_extraction_service import metadata_extraction_service
from app.services.text_cleaning_service import get_text_cleaning_service
from app.search_indexes.hnsw import HNSWBuilder

[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1010)>
2025-07-17 15:24:17,002 | INFO | Loading KenLM 5-gram model...
2025-07-17 15:24:17,003 | INFO | Using cached HuggingFace KenLM model: app/models/models--BramVanroy--kenlm_wikipedia_en/snapshots/744f3005d30510c09ae7302baf62d3cdc0a2bdec/wiki_en_token.arpa.bin
2025-07-17 15:24:20,069 | INFO | KenLM model loaded successfully.
2025-07-17 15:24:20,069 | INFO | Loading spaCy model...
2025-07-17 15:24:20,270 | INFO | spaCy model loaded successfully.
2025-07-17 15:24:20,271 | INFO | Loading SymSpell model...
2025-07-17 15:24:20,659 | INFO | SymSpell model loaded successfully.


In [2]:
from pathlib import Path
text = document_conversion_service.process_document(Path("test_data/House Rules.pdf"))

2025-07-17 15:24:20,670 | INFO | Processing document: test_data/House Rules.pdf
2025-07-17 15:24:20,673 | INFO | detected formats: [<InputFormat.PDF: 'pdf'>]
2025-07-17 15:24:20,712 | INFO | Going to convert document batch...
2025-07-17 15:24:20,712 | INFO | Initializing pipeline for StandardPdfPipeline with options hash 0bc13843d63324263b53bf2972b9f09a
2025-07-17 15:24:20,732 | INFO | Loading plugin 'docling_defaults'
2025-07-17 15:24:20,734 | INFO | Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-07-17 15:24:20,917 | INFO | Accelerator device: 'mps'
2025-07-17 15:24:20,919 - OrtInferSession - WARNING: DirectML is only supported in Windows OS. The current OS is Darwin. Use CoreMLExecutionProvider inference by default.
2025-07-17 15:24:20,919 | WARNING | DirectML is only supported in Windows OS. The current OS is Darwin. Use CoreMLExecutionProvider inference by default.
2025-07-17 15:24:20,957 - OrtInferSession - WARNING: DirectML is only suppor

In [ ]:
print(text)

In [3]:
textCleaner = get_text_cleaning_service()

In [4]:
cleanedText = textCleaner.clean_document_with_placeholders(text)

In [ ]:
print(cleanedText)

In [5]:
chunks = await chunking_service.chunk_document(text)
print(f"Total chunks: {len(chunks)}")

2025-07-17 15:24:54,073 | INFO | Starting document chunking for text of length 17526
2025-07-17 15:24:54,090 | INFO | Document complexity: 0.81, target chunk size: 300 words
2025-07-17 15:24:54,091 | INFO | Created 7 initial chunks with target size 300 words
2025-07-17 15:24:57,493 | INFO | HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-07-17 15:24:57,498 | INFO | LLM chunking response: Based on the instructions and the document text, I suggest the following splits:

split_after: 0, 1, 2, 3, 5
2025-07-17 15:24:57,499 | INFO | Split after chunks: [0, 1, 2, 3, 5]
2025-07-17 15:24:57,501 | INFO | Found 7 chunks
2025-07-17 15:24:57,501 | INFO | Created 6 sections
2025-07-17 15:24:57,502 | INFO | Successfully chunked document into 6 chunks


Total chunks: 6


In [8]:
chunks_with_context = await context_generation_service.create_contextualized_chunks(chunks, text)

2025-07-17 15:26:46,459 | INFO | Generating contexts for 6 chunks
2025-07-17 15:26:46,460 | INFO | Processing chunk 1/6
2025-07-17 15:26:51,851 | INFO | HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-07-17 15:26:51,852 | INFO | Processing chunk 2/6
2025-07-17 15:26:57,110 | INFO | HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-07-17 15:26:57,115 | INFO | Processing chunk 3/6
2025-07-17 15:27:01,425 | INFO | HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-07-17 15:27:01,426 | INFO | Processing chunk 4/6
2025-07-17 15:27:06,050 | INFO | HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-07-17 15:27:06,052 | INFO | Processing chunk 5/6
2025-07-17 15:27:15,064 | INFO | HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-07-17 15:27:15,066 | INFO | Processing chunk 6/6
2025-07-17 15:27:19,976 | INFO | HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK

In [ ]:
print(chunks_with_context[3])

In [41]:
embeddings = embedding_service.generate_embeddings_batch(chunks_with_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
metadata_list = []
for i, chunk in enumerate(chunks):
    #context = contexts[i] if i < len(contexts) else ""
    #metadata = metadata_extraction_service.extract_metadata(chunk, context)
    metadata = {
            "keywords": [],
            "topics": [],
            "entities": [],
            "document_type": "unknown"
        }
    # Optionally, add extra fields, e.g., for HNSW filtering
    metadata['chunk_index'] = i
    metadata_list.append(metadata)

In [16]:
from uuid import uuid4

document_id = str(uuid4())
chunk_ids = [f"{document_id}_chunk_{i}" for i in range(len(chunks_with_context))]
document_ids = [document_id] * len(chunks_with_context)

In [44]:
builder = HNSWBuilder(organization_id="test-org", M=8, ef_construction=50, seed=42)
index = builder.build_index(
    vectors=embeddings,
    chunk_ids=chunk_ids,
    document_ids=document_ids,
    metadata_list=metadata_list,
)

2025-07-17 17:29:25,268 | INFO | Building HNSW index for organization test-org with 6 vectors
2025-07-17 17:29:25,271 | INFO | HNSW index construction complete. Added 6 nodes.


In [45]:
query = "Is there a limit to how many people can live in an apartment at University Court"
query_vec = embedding_service.generate_embedding(query)

results = index.search(query_vec, k=5)
for dist, node_id, chunk_id in results:
    idx = chunk_ids.index(chunk_id)
    print(f"Score: {1-dist:.3f} | Chunk: {chunks[idx]}...")
    print("Keywords:", metadata_list[idx].get("keywords", []))
    print()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Score: 0.434 | Chunk: 3. c)Only authorized storage sheds approved by the Board are allowed on the lanais.Unless hidden from sight by railingfabric,all other storageunitsarenot allowed withoutwritten approval fromtheBoard. <!-- page_break -->

- 6)Plants.All plants shall be placed in containers with drip pans to prevent thedrippingofwaterorsoil ontoother apartmentsorthe common elements.Care should be taken in scrubbing lanais so astopreventwater fromrunningdown theexteriorof the building. - 7)Barbecuing.No fires orbarbecuing is permitted on any apartment lanai.Barbecuing ispermitted in the courtyard/patio areaorother areaspermittedby theBoardorManagingAgent. - 8)Throwing Objects from Building. Nothing shall be thrown from lanais,windows,entrancebalconies,etc.-Cigarettes and matches,specifically,arefirehazard.The throwingof firecrackers from lanaisand the explosionofanyfireworks anywhereon thegroundsorwithinthebuildingisprohibited
- 9)Waterbeds.No waterbeds are permitted in the apartment

In [23]:
from app.search_indexes.bm25 import BM25Index

In [28]:
bm25_index = BM25Index(organization_id='alkalskjdfh')

In [29]:
document_id = str(uuid4())
chunk_tuples = [
    (
        f"{document_id}_chunk_{i}",   # chunk_id
        chunk,                        # content (the chunk text)
        document_id,                  # document_id
        {}                            # metadata (add any fields if you want)
    )
    for i, chunk in enumerate(chunks_with_context)
]

In [39]:
for chunk in chunk_tuples:
    if chunk[0] == "59e23eec-b77f-4441-9c87-16cb392ebdba_chunk_4":
        print(chunk[1])

<chunk_context>This section of the document outlines rules for the University Court condominium project, specifically focusing on the maintenance and upkeep of apartments, common areas, and parking spaces. It emphasizes the importance of responsible behavior and cleanliness, as well as the need for prior approval from the Board or Managing Agent for certain activities, such as installing storage sheds or using hazardous materials. The section also touches on the maintenance of apartments, including repairs and structural changes, as well as the responsibilities of owners and tenants.</chunk_context>
<chunk>3. c)Only authorized storage sheds approved by the Board are allowed on the lanais.Unless hidden from sight by railingfabric,all other storageunitsarenot allowed withoutwritten approval fromtheBoard. <!-- page_break -->

- 6)Plants.All plants shall be placed in containers with drip pans to prevent thedrippingofwaterorsoil ontoother apartmentsorthe common elements.Care should be taken